In [ ]:
# !pip install python-dotenv

In [ ]:
# !pip install sentence-transformers
# !pip install huggingface_hub[hf_xet]

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv("./.env_gemini")
api_key = os.getenv('GEMINI_API_KEY')

In [10]:
import os
from typing import List
import numpy as np
import pdfplumber
import faiss
from sentence_transformers import SentenceTransformer
from google import genai
from dotenv import load_dotenv


# =========================
# 환경 설정
# =========================
load_dotenv("./.env_gemini")
api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    raise EnvironmentError("환경변수 api_key 가 설정되어 있지 않습니다.")

PDF_DIR = "./data"  # ✅ 폴더 안 모든 PDF를 읽기
GENERATION_MODEL = "gemini-2.5-flash"   # 답변 생성용

# =========================
# 1) 모델 초기화 (Gemini + SentenceTransformer)
# =========================
client = genai.Client(api_key=api_key)

embed_model = SentenceTransformer(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

# =========================
# 2) ./pdf 안의 모든 PDF 텍스트 로딩 + 출력 + 청크 분할
# =========================
print("[RAG] PDF 디렉터리 스캔 중...")

all_chunks: List[str] = []

max_chars = 500
overlap_chars = 150

# 디렉터리 내 PDF 파일 리스트
pdf_files = [
    f for f in os.listdir(PDF_DIR)
    if f.lower().endswith(".pdf")
]

if not pdf_files:
    raise FileNotFoundError(f"{PDF_DIR} 안에 PDF 파일이 없습니다.")

for pdf_file in pdf_files:
    pdf_path = os.path.join(PDF_DIR, pdf_file)
    print(f"\n[RAG] PDF 로딩 중... {pdf_file}")
    all_text = ""

    # ---- PDF 전체 텍스트 추출 ----
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text()
            if text:
                all_text += text + "\n\n"

    all_text = all_text.strip()
    if not all_text:
        print(f"[경고] {pdf_file} 에서 텍스트를 추출하지 못했습니다. 건너뜀.")
        continue

    print("\n====== [PDF 전체 내용 출력 시작] ======\n")
    print(len(all_text))
    print("\n====== [PDF 전체 내용 출력 끝] ======\n")

    # ---- 텍스트 청크 분할 ----
    chunks: List[str] = []
    if len(all_text) <= max_chars:
        chunks = [all_text]
    else:
        start = 0
        while start < len(all_text):
            end = start + max_chars
            chunk = all_text[start:end].strip()
            if chunk:
                chunks.append(chunk)
            start = end - overlap_chars  # overlap 적용

    print(f"[RAG] {pdf_file} 청크 분할 완료 - 청크 개수: {len(chunks)}")

    # 전체 코퍼스에 추가
    all_chunks.extend(chunks)

# 디렉터리 내 모든 PDF에서 청크가 생성되었는지 확인
if not all_chunks:
    raise ValueError("어떤 PDF에서도 유효한 텍스트 청크를 만들지 못했습니다.")

print(f"\n[RAG] 전체 청크 개수 (모든 PDF 합산): {len(all_chunks)}")

# =========================
# 3) 청크 임베딩 생성
# =========================
print("[RAG] 청크 임베딩 생성 중... (SentenceTransformer)")
vectors = embed_model.encode(all_chunks, convert_to_numpy=True, show_progress_bar=False)

# =========================
# 4) FAISS 인덱스 구축 (코사인 유사도)
# =========================
print("[RAG] FAISS 인덱스 구축 중...")
texts = all_chunks  # 인덱스와 함께 저장할 텍스트 리스트

dim = vectors.shape[1]
index = faiss.IndexFlatIP(dim)

# L2 normalize -> 내적 = 코사인 유사도
norms = np.linalg.norm(vectors, axis=1, keepdims=True)
vecs_norm = vectors / (norms + 1e-10)
index.add(vecs_norm.astype("float32"))

print("[RAG] 인덱스 구축 완료!")

# =========================
# 5) 질의 루프 (검색 + 생성)
# =========================
print("\n=== RAG 질의 데모 (FAISS + 로컬 임베딩 + Gemini 생성) ===")
print("현재 인덱스는 ./pdf 폴더 안의 모든 PDF를 기반으로 합니다.")
print("종료하려면 'q' 입력\n")

while True:
    query = input("질문을 입력하세요: ")
    if query.lower().strip() in {"q", "quit", "exit"}:
        break

    # 5-1) 질의 임베딩
    q_vec = embed_model.encode([query], convert_to_numpy=True, show_progress_bar=False)[0]
    q_vec = q_vec.reshape(1, -1)

    # L2 normalize
    q_norms = np.linalg.norm(q_vec, axis=1, keepdims=True)
    q_vec_norm = q_vec / (q_norms + 1e-10)

    # 5-2) FAISS 검색
    k = 50  # ✅ 먼저 넉넉하게 가져오기
    distances, indices = index.search(q_vec_norm.astype("float32"), k)

    # 상위 k개 텍스트에서, 전체 글자수 제한 두고 사용
    idxs = indices[0]
    contexts: List[str] = []
    max_context_chars = 20000  # ✅ Gemini에 보낼 전체 컨텍스트 길이
    current_len = 0

    for i in idxs:
        if i == -1:
            continue
        chunk_text = texts[i]
        if current_len + len(chunk_text) > max_context_chars:
            break
        contexts.append(chunk_text)
        current_len += len(chunk_text)

    context_block = "\n\n---\n\n".join(contexts)

    # 5-3) 프롬프트 구성
    system_prompt = (
        "너는 RAG 질문-응답 봇이다. 아래 '컨텍스트'의 정보만 사용해서 "
        "질문에 정확하고 간결하게 한국어로 답변해라.\n"
        "- 컨텍스트에 없는 정보는 절대 추측하지 말아라.\n"
        "- 모르는 정보가 있으면 '이 PDF에 있는 정보만으로는 알 수 없습니다.'라고 답해라.\n"
        "- 금리, 기간 등 숫자는 반드시 컨텍스트에 나온 값만 사용해라.\n"
        "- 총 이자 같은 것은 정확히 계산할 수 있을 경우 답변해도 됩니다.\n"
        "- 나이, 성별 같은 기본 정보를 주면 보험료를 계산해서 답변해도 됩니다.\n"
        "- 질문과 비슷한 내용이나 질문 내용을 포함한 항목이 있으면 답변해줘\n"
        
    )

    full_prompt = f"""{system_prompt}

[컨텍스트 시작]
{context_block}
[컨텍스트 끝]

[질문]
{query}
"""

    # 5-4) Gemini 호출
    resp = client.models.generate_content(
        model=GENERATION_MODEL,
        contents=full_prompt,
    )

    # 5-5) 응답 텍스트 추출
    if getattr(resp, "text", None):
        answer_text = resp.text.strip()
    else:
        collected = []
        for cand in getattr(resp, "candidates", []) or []:
            content = getattr(cand, "content", None)
            if not content:
                continue
            for part in getattr(content, "parts", []) or []:
                txt = getattr(part, "text", None)
                if txt:
                    collected.append(txt.strip())
        if collected:
            answer_text = "\n".join(collected)
        else:
            answer_text = "모델이 유효한 텍스트 응답을 반환하지 않았습니다."

    # 5-6) 출력
    print("\n[답변]")
    print(answer_text)
    print("\n" + "=" * 60 + "\n")


[RAG] PDF 디렉터리 스캔 중...

[RAG] PDF 로딩 중... 약관.pdf

====== [PDF 전체 내용 출력 시작] ======

268016

====== [PDF 전체 내용 출력 끝] ======

[RAG] 약관.pdf 청크 분할 완료 - 청크 개수: 766

[RAG] 전체 청크 개수 (모든 PDF 합산): 766
[RAG] 청크 임베딩 생성 중... (SentenceTransformer)
[RAG] FAISS 인덱스 구축 중...
[RAG] 인덱스 구축 완료!

=== RAG 질의 데모 (FAISS + 로컬 임베딩 + Gemini 생성) ===
현재 인덱스는 ./pdf 폴더 안의 모든 PDF를 기반으로 합니다.
종료하려면 'q' 입력

질문을 입력하세요: 보험 가입 조건 알려줘


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash\nPlease retry in 47.395778846s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerDayPerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-2.